In [14]:
import numpy as np
import pandas as pd

In [36]:
# read DFA from csv
inputFile= "C:/Users/Admin/Desktop/FA CSV input files/DFA6.csv"
DFA = pd.read_csv(inputFile, index_col="states") 
# seting data type
DFA.index = DFA.index.map(str)
DFA.astype(str)

#
print("Given DFA")
print(DFA)

Given DFA
       input_a input_b  isInitial  isFinal
states                                    
q0          q2      q1       True     True
q1          q2      q0      False    False
q2          q1      q2      False    False
p0          p3      p1      False     True
p1          p3      p0      False    False
p2          p3      p0      False    False
p3          p2      p3      False    False


In [26]:
#identify final state
states=DFA.index.values.tolist()
finalStates=DFA.loc[DFA["isFinal"]==True].index.values.tolist()
#print(states)
print("finalStates:", finalStates)

finalStates: ['q0', 'p0']


In [27]:
#initialize table
table= pd.DataFrame(index=set([frozenset([str(a),str(b)]) for a in DFA.index.values.tolist() for b in DFA.index.values.tolist() if a!=b]), columns= ["eq"])


In [28]:
# base case
table.loc[[frozenset([a,b]) for a in finalStates for b in states if b not in finalStates ]]= 0
print("Step 0 \nFollowing pairs are distinguishable in base case")
print(*[set([a,b]) for a in finalStates for b in states if b not in finalStates ], sep="\n")


Step 0 
Following pairs are distinguishable in base case
{'q0', 'q1'}
{'q0', 'q2'}
{'q0', 'p1'}
{'p2', 'q0'}
{'q0', 'p3'}
{'q1', 'p0'}
{'q2', 'p0'}
{'p0', 'p1'}
{'p2', 'p0'}
{'p0', 'p3'}


In [29]:
keepgoing=True
iteration=1
while keepgoing:
    print("Step",iteration)
    symbols= [x for x in list(DFA.columns) if x.find("input")!=-1]
    #print([x for x in table[table['eq'].isnull()].index.values.tolist()])
    #print()
    pairstoupdate=[]
    
    for s in symbols:
        #print(s)
        pairstoupdate=pairstoupdate+[ frozenset(list(x))
               for x in table[table['eq'].isnull()].index.values.tolist()
               if DFA.loc[[list(x)[0]],s].values[0]!=DFA.loc[[list(x)[1]],s].values[0]
               and table.loc[[frozenset((str(DFA.loc[[list(x)[0]],s].values[0]),str(DFA.loc[[list(x)[1]],s].values[0])))], "eq"].values[0] >=0

              ] 
        print("Following pairs are distinguishable due to "+s +" at this step")
        print(*[ str(set(x))+ " as tran("+ str(set(x))+" , " +s+")= {"+str(DFA.loc[[list(x)[0]],s].values[0])
               +" , "+ str(DFA.loc[[list(x)[1]],s].values[0]) +"}" 
               
               for x in table[table['eq'].isnull()].index.values.tolist()
               if DFA.loc[[list(x)[0]],s].values[0]!=DFA.loc[[list(x)[1]],s].values[0]
               and table.loc[[frozenset((str(DFA.loc[[list(x)[0]],s].values[0]),str(DFA.loc[[list(x)[1]],s].values[0])))], "eq"].values[0] >=0

              ],sep='\n')
    
    if len(pairstoupdate)==0:
        break
    table.loc[pairstoupdate]= iteration
    iteration=iteration+1
#print(table)

    

Step 1
Following pairs are distinguishable due to input_a at this step

Following pairs are distinguishable due to input_b at this step
{'p2', 'p3'} as tran({'p2', 'p3'} , input_b)= {p0 , p3}
{'q1', 'p3'} as tran({'q1', 'p3'} , input_b)= {q0 , p3}
{'p3', 'p1'} as tran({'p3', 'p1'} , input_b)= {p3 , p0}
{'q2', 'q1'} as tran({'q2', 'q1'} , input_b)= {q2 , q0}
{'q2', 'p1'} as tran({'q2', 'p1'} , input_b)= {q2 , p0}
{'p2', 'q2'} as tran({'p2', 'q2'} , input_b)= {p0 , q2}
Step 2
Following pairs are distinguishable due to input_a at this step

Following pairs are distinguishable due to input_b at this step



In [30]:
# print table
print("table")
for i in range (1,len(DFA.index.values)):
    print(DFA.index.values[i],[x if x>-1 else ""  for x in  table.loc[[frozenset([x,DFA.index.values[i]]) for x in DFA.index.values[range(0,i)]], "eq"].values.tolist()])
 
print(states[0:len(states)-1])

# print all equivalent states
print("\nFollowing states are equivalent")
[set(x) for x in table[table["eq"].isnull()].index.values.tolist()]

table
q1 [0]
q2 [0, 1]
p0 ['', 0, 0]
p1 [0, '', 1, 0]
p2 [0, '', 1, 0, '']
p3 [0, 1, '', 0, 1, 1]
['q0', 'q1', 'q2', 'p0', 'p1', 'p2']

Following states are equivalent


[{'p0', 'q0'}, {'p2', 'q1'}, {'p1', 'q1'}, {'p3', 'q2'}, {'p1', 'p2'}]

In [31]:
#define states of mimibal DFA by merging equivalent states
newStates=[] # a list of sets
oldStates=states.copy()

for x in  table[table["eq"].isnull()].index.values.tolist():
    
    temp=""
    
    p=list(x)[0]
    q=list(x)[1]
    # if both are in the list
    if p in oldStates and q in oldStates: 
        temp=str(q)+" "+str(list(x)[0])
        
        del oldStates[oldStates.index(list(x)[0])]
        del oldStates[oldStates.index(q)]
        newStates.insert(1,temp.strip())
        
        
    # if 1st in in list and 2nd is not    
    
    if p in oldStates and q not in oldStates: 
        # find 2nd in new list
        indx= newStates.index([x for x in newStates if x.find(q)!=-1][0])
        # merge this in the new list 
        temp= newStates[indx]
        temp= temp+" "+str(p) 
        #delete 1st from old list
        del oldStates[oldStates.index(p)]
        # delete previous state from newStates
        del newStates[indx]
        newStates.insert(1,temp.strip())
        
   # if 2ndin in list and 2nd is not    
    if p not in oldStates and q  in oldStates: 
        # find 1st in new list 
        indx= newStates.index([x for x in newStates if x.find(p)!=-1][0])
        # merge this in the new list 
        temp= newStates[indx]
        temp= temp+" "+str(q) 
        #delete 2nd from old list
        del oldStates[oldStates.index(q)]
        del newStates[indx]
        newStates.insert(1,temp.strip())
   
newStates=newStates+oldStates
#print("newStates:",newStates)




In [32]:
# creating minimal DFA is pandas dataframe and finding transitions for new states 
mDFA= pd.DataFrame(index=newStates, columns= list(DFA.columns))

    #print(s)
for x in newStates:  
    #print("state:",x)      
    outputstate=str(DFA.loc[x.split()[0], s])
    #print("output state:",outputstate)        
    out= [x for x in newStates if x.find(outputstate)!=-1][0]
    #print (out)
    mDFA.loc[x, s]= out
    #print(mDFA.loc[x])
        
#print(mDFA)

In [33]:
#set initial states
intialState=DFA.loc[DFA["isInitial"]==True].index.values[0]
#print(intialState)
newIniS= [x for x in newStates if x.find(intialState)!=-1][0]
#print(newIniS)
mDFA.loc[newIniS, "isInitial"]= True


# mark finalstates
newFinalStates= [x for x in newStates if len(set(finalStates).intersection(set(x.split())))>0]

mDFA.loc[newFinalStates, "isFinal"]= True
#print(mDFA)

In [34]:
# print output
print("minimized DFA")
print(mDFA)

minimizaed DFA
         input_a   input_b isInitial isFinal
p0 q0        NaN  q1 p2 p1      True    True
p3 q2        NaN     p3 q2       NaN     NaN
q1 p2 p1     NaN     p0 q0       NaN     NaN
